In [1]:
import tensorflow as tf
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanSquaredError
import gymnasium as gym
import tensorflow as tf
import numpy as np
import json
import random

import dqn.modules.rewards as dqn_rewards
import dqn.modules.processing as dqn_processing

In [2]:
custom_objects = { 
    'mse': MeanSquaredError()
}

# Load the model architecture with custom objects
model = tf.keras.models.load_model('dqn/models/v_20.2/ep_400_model.h5', custom_objects=custom_objects)

# Load the model weights
model.load_weights('dqn/models/v_20.2/ep_400_model.h5')

# Verify the model
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)             │ (None, 13)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │         3,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,933 (175.52 KB)

 Trainable params: 44,931 (175.51 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [3]:
with open('dqn/parameters/v_20.2/parameters_v_20.2.json', 'r') as file:
    params = json.load(file)

In [4]:
MODEL_VERSION = params["MODEL_VERSION"]
EVIROMENT_VARIANT = params["EVIROMENT_VARIANT"]
obs_type = params["obs_type"]
DIFFICULTY = params["DIFFICULTY"]
MODE = params["MODE"]
STATE_SIZE = params["STATE_SIZE"]
ACTIONS_SIZE = params["ACTIONS_SIZE"]
ACTIONS = params["ACTIONS"]
LEARNING_RATE = params["LEARNING_RATE"]
EPSILON = params["EPSILON"]
EPSILON_MIN = params["EPSILON_MIN"]
EPSILON_DECAY_RATE = params["EPSILON_DECAY_RATE"]
GAMMA = params["GAMMA"]
RENDER_INTERVAL = params["RENDER_INTERVAL"]
TOTAL_EPISODES = params["TOTAL_EPISODES"]
REPLAY_INTERVAL = params["REPLAY_INTERVAL"]
MINI_BATCHES_REPLAY = params["MINI_BATCHES_REPLAY"]
REPLAY_BUFFER_MEMORY = params["REPLAY_BUFFER_MEMORY"]
MINIMUM_REPLAY_HISTORY = params["MINIMUM_REPLAY_HISTORY"]
AVERAGE_WINDOW = params["AVERAGE_WINDOW"]
UPDATE_TARGET_MODEL_FREQUENCY = params["UPDATE_TARGET_MODEL_FREQUENCY"]
MODEL_SAVE_INTERVALL = params["MODEL_SAVE_INTERVALL"]

: 

In [5]:
MODE = 0 #random.randint(0, 7   )

env = gym.make(EVIROMENT_VARIANT, obs_type="ram", render_mode="human", difficulty=1, mode=MODE) 

state = env.reset()[0]
preprocessed_state = dqn_processing.preprocess_ram(state, path="dqn/logs/sample_frame.txt")
total_reward = 0
y_pos_prev = 0
average_rewards = []
rewards_per_episode = []
distance_per_episode = []
max_distances = []
game_done = False

while not game_done:
    # take random action action
    pred =model.predict(preprocessed_state, verbose=0)
    print(pred, np.argmax(pred))
    action = np.argmax(pred)
    next_state, game_reward, game_done, game_loss_of_live, game_info = env.step(action)

    y_pos = env.ale.getRAM()[14] // 3
    crashed = 1 if env.ale.getRAM()[16] != 255 else 0 # RAM(16) =:= Collision Lane

    next_state_preprocessed = dqn_processing.preprocess_ram(next_state, path="dqn/logs/sample_frame.txt")

    # update reward
    total_reward, gained_reward = dqn_rewards.action_based_reward(total_reward, crashed, action, y_pos, y_pos_prev, game_reward)


    # set the next state to the current state
    preprocessed_state = next_state_preprocessed
    y_pos_prev = y_pos

[[3.3817635 3.7727482 3.2418354]] 1
[[53.0758   56.10315  52.060745]] 1


c:\Users\Jan\anaconda3\envs\FroggerEnv\lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.ale to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.ale` for environment variables or `env.get_wrapper_attr('ale')` that will search the reminding wrappers.
  logger.warn(


[[51.951153 54.91558  50.959064]] 1
[[50.84073  53.748096 49.875862]] 1
[[49.835423 52.688507 48.894096]] 1
[[48.735996 51.53034  47.81897 ]] 1
[[47.53967 50.26798 46.65088]] 1
[[46.324398 48.987255 45.46612 ]] 1
[[45.427174 48.038235 44.588787]] 1
[[44.060772 46.599503 43.255413]] 1
[[43.0796   45.564404 42.298176]] 1
[[42.077736 44.506165 41.318954]] 1
[[41.15359  43.5335   40.417065]] 1
[[40.163235 42.48512  39.448395]] 1
[[39.230824 41.500996 38.536175]] 1
[[38.217937 40.432926 37.548046]] 1
[[37.20898  39.36971  36.562668]] 1
[[36.310272 38.416542 35.681988]] 1
[[35.221478 37.271877 34.618572]] 1
[[34.330997 36.33387  33.748016]] 1
[[33.431587 35.387074 32.866802]] 1
[[32.529556 34.43707  31.982944]] 1
[[31.629562 33.487682 31.103258]] 1
[[30.806887 32.620564 30.295654]] 1
[[29.89114  31.655231 29.397867]] 1
[[29.063524 30.784828 28.587227]] 1
[[28.33724  30.016613 27.875103]] 1
[[27.633005 29.277704 27.183308]] 1
[[26.751879 28.346972 26.320486]] 1
[[26.208141 27.774527 25.785246